# SensRay Basic Usage Demo

This notebook demonstrates the core SensRay functionality:
- Loading standard Earth models (PREM, AK135, etc.)
- Creating tetrahedral meshes
- Populating seismic properties
- Basic visualization and cross-sections

In [ ]:
import numpy as np
from sensray import PlanetModel, CoordinateConverter
import matplotlib.pyplot as plt

## 1. Load a Standard Earth Model

SensRay includes standard models like PREM, AK135, IASP91, etc.

In [ ]:
# Load PREM model
model = PlanetModel.from_standard_model('prem')

# Show available models
print("Available models:", PlanetModel.list_standard_models()[:5], "...")
print(f"Model radius: {model.radius:.1f} km")
print(f"Discontinuities at depths: {list(model.get_discontinuities(as_depths=True).keys())[:4]} km (first 4)")

In [ ]:
# Plot 1D velocity profiles
fig, ax = model.plot_profiles(properties=['vp', 'vs'], max_depth_km=6371)
plt.title('PREM Velocity Profiles')
plt.show()

                "## 2. Create Tetrahedral Meshes\n",
                "\n",
                "SensRay uses tetrahedral meshes for accurate ray tracing through complex Earth models."

In [ ]:
# Create tetrahedral mesh
mesh = model.create_mesh(
    mesh_type='tetrahedral',
    mesh_size_km=500.0,
    populate_properties=['vp', 'vs', 'rho']
)
print(f"Tetrahedral mesh: {mesh.mesh.n_cells} cells, {mesh.mesh.n_points} points")

In [ ]:
# Mesh properties are automatically populated from the 1D model
print(f"Properties populated: {list(mesh.mesh.cell_data.keys())[-3:]}")

## 3. Visualize Properties

Create cross-sections and spherical shells to visualize seismic properties.

In [ ]:
# Cross-section through the equator (y=0 plane)
plotter = mesh.plot_cross_section(
    plane_normal=(0, 1, 0),  # y=0 plane
    property_name='vp',
    show_edges=True
)
plotter.camera.position = (10000, 8000, 8000)
plotter.show()

In [ ]:
# Spherical shell at 670 km depth (upper-lower mantle boundary)
shell_radius = model.radius - 670  # 670 km depth
plotter = mesh.plot_spherical_shell(
    radius_km=shell_radius,
    property_name='vs',
    show_edges=False
)
plotter.show()

## 4. Inspect Mesh Properties

Use built-in tools to examine what's stored on the mesh.

In [ ]:
# List all properties with statistics
info = mesh.list_properties(show_stats=True)

# Show property ranges
vp_values = mesh.mesh.cell_data['vp']
print(f"\nVp range: {vp_values.min():.2f} - {vp_values.max():.2f} km/s")
print(f"Mean Vp: {vp_values.mean():.2f} km/s")

## 5. Save and Load Meshes

Meshes can be saved to VTU format with metadata for later use.

In [ ]:
# Save mesh with all properties
mesh.save('prem_tet_demo')
print("Saved mesh to prem_tet_demo.vtu and prem_tet_demo_metadata.json")

# Load mesh back
loaded_mesh = model.create_mesh(from_file='prem_tet_demo')
print(f"Loaded mesh: {loaded_mesh.mesh.n_cells} cells")
print(f"Available properties: {list(loaded_mesh.mesh.cell_data.keys())}")

## Summary

This demo showed:
- Loading standard Earth models
- Creating tetrahedral meshes
- Populating seismic properties from 1D models
- Visualizing cross-sections and spherical shells
- Saving/loading meshes for reuse

Next: See `02_ray_tracing_kernels.ipynb` for ray tracing and sensitivity kernel computation.